# Открытый курс машинного обучения. Тема 4. Линейные модели классификации и регрессии
## [habr](https://habr.com/ru/companies/ods/articles/323890/)
## [YouTube](https://www.youtube.com/watch?v=NO4KSNbsXZE)